# Pokemon Classifier Training Notebook

Train a MobileNetV3-based classifier to identify Gen 1 Pokemon (151 classes).

**Run this notebook in Google Colab for GPU acceleration.**

**Note:** This notebook uses local Colab storage. Files will not persist between sessions.
Make sure to download your trained model before the session ends!

In [ ]:
#@title 1. Setup and Configuration
#@markdown Install dependencies and configure paths (no Google Drive required)

# Install TensorFlow and Kaggle first
!pip install -q tensorflow kaggle

# Upgrade packaging to satisfy Colab's pre-installed packages (bigquery, xarray, etc.)
!pip install -q --upgrade packaging

# Install tensorflowjs with --no-deps to avoid packaging version conflict
# (tensorflowjs requires packaging~=23.1 but Colab needs >=24.x)
# The actual runtime dependencies are already satisfied by tensorflow
!pip install -q --no-deps tensorflowjs

# Configuration
CONFIG = {
    # Paths (local Colab storage - does not persist between sessions)
    'data_dir': '/content/data',
    'models_dir': '/content/models',
    
    # Dataset
    'kaggle_dataset': 'lantian773030/pokemonclassification',
    
    # Image settings
    'image_size': 224,
    'batch_size': 32,
    
    # Training
    'epochs_frozen': 10,
    'epochs_unfrozen': 15,
    'learning_rate_frozen': 1e-3,
    'learning_rate_unfrozen': 1e-5,
    'dropout_rate': 0.5,
    'validation_split': 0.15,
    'test_split': 0.15,
    
    # Fine-tuning
    'unfreeze_layers': 20,
    
    # Augmentation
    'rotation_range': 20,
    'zoom_range': 0.15,
    'horizontal_flip': True,
    
    # Callbacks
    'early_stopping_patience': 5,
    'reduce_lr_patience': 3,
}

import os
os.makedirs(CONFIG['data_dir'], exist_ok=True)
os.makedirs(CONFIG['models_dir'], exist_ok=True)

print("Configuration complete")
print(f"  Data directory: {CONFIG['data_dir']}")
print(f"  Models directory: {CONFIG['models_dir']}")
print("")
print("IMPORTANT: Files are stored locally and will be deleted when the session ends.")
print("Make sure to download your trained model before disconnecting!")

In [ ]:
#@title 2. Download Dataset from Kaggle
#@markdown Enter your Kaggle credentials below (Get from: https://www.kaggle.com/settings/account → API)
#@markdown
#@markdown **OR** skip this cell and run Cell 2b to generate a fresh official dataset from PokeAPI

KAGGLE_USERNAME = ""  #@param {type:"string"}
KAGGLE_KEY = ""  #@param {type:"string"}

import os
from pathlib import Path
import json

# Check if dataset already exists
data_path = Path(CONFIG['data_dir'])
existing_dirs = list(data_path.glob("*/"))

if len(existing_dirs) > 100:
    print(f"Dataset already downloaded ({len(existing_dirs)} directories found)")
else:
    # Validate credentials
    if not KAGGLE_USERNAME or not KAGGLE_KEY:
        print("ERROR: Please fill in your Kaggle credentials above!")
        print("")
        print("How to get your credentials:")
        print("  1. Go to https://www.kaggle.com/settings/account")
        print("  2. Scroll to 'API' section")
        print("  3. Click 'Create New Token'")
        print("  4. Download kaggle.json file")
        print("  5. Open it and copy the username and key values")
        print("  6. Paste them into KAGGLE_USERNAME and KAGGLE_KEY above")
        print("")
        print("ALTERNATIVE: Skip this cell and run Cell 2b to generate")
        print("             a fresh dataset from official PokeAPI sources!")
        raise ValueError("Kaggle credentials required")
    
    # Create Kaggle credentials
    os.makedirs('/root/.kaggle', exist_ok=True)
    kaggle_json = {
        "username": KAGGLE_USERNAME,
        "key": KAGGLE_KEY
    }
    
    with open('/root/.kaggle/kaggle.json', 'w') as f:
        json.dump(kaggle_json, f)
    os.chmod('/root/.kaggle/kaggle.json', 0o600)
    
    print("Kaggle credentials configured")
    print(f"  Username: {KAGGLE_USERNAME}")
    print("")
    
    # Download dataset
    print("Downloading dataset...")
    !kaggle datasets download -d {CONFIG['kaggle_dataset']} -p {CONFIG['data_dir']} --unzip
    
    print("Dataset downloaded")

# Verify dataset
def count_images(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len([f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    return count

total_images = count_images(CONFIG['data_dir'])
print(f"  Total images: {total_images}")

In [ ]:
#@title 2b. Generate Official Dataset (Alternative to Kaggle)
#@markdown **ALTERNATIVE**: Generate a fresh dataset from official PokeAPI sources
#@markdown
#@markdown This downloads official Pokemon sprites and artwork, then augments them
#@markdown to create a high-quality training dataset. No Kaggle credentials needed!
#@markdown
#@markdown Benefits:
#@markdown - Clean, verified labels (from official Pokemon API)
#@markdown - Multiple sprite variants (official artwork, game sprites, shiny forms)
#@markdown - Consistent image quality
#@markdown - ~6000+ images after augmentation

import os
import sys
import time
import json
import hashlib
import requests
from pathlib import Path
from io import BytesIO

# Install Pillow for image processing
!pip install -q Pillow

from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import random

# Gen 1 Pokemon list (151 Pokemon)
GEN1_POKEMON = [
    "bulbasaur", "ivysaur", "venusaur", "charmander", "charmeleon", "charizard",
    "squirtle", "wartortle", "blastoise", "caterpie", "metapod", "butterfree",
    "weedle", "kakuna", "beedrill", "pidgey", "pidgeotto", "pidgeot",
    "rattata", "raticate", "spearow", "fearow", "ekans", "arbok",
    "pikachu", "raichu", "sandshrew", "sandslash", "nidoran-f", "nidorina",
    "nidoqueen", "nidoran-m", "nidorino", "nidoking", "clefairy", "clefable",
    "vulpix", "ninetales", "jigglypuff", "wigglytuff", "zubat", "golbat",
    "oddish", "gloom", "vileplume", "paras", "parasect", "venonat", "venomoth",
    "diglett", "dugtrio", "meowth", "persian", "psyduck", "golduck",
    "mankey", "primeape", "growlithe", "arcanine", "poliwag", "poliwhirl",
    "poliwrath", "abra", "kadabra", "alakazam", "machop", "machoke", "machamp",
    "bellsprout", "weepinbell", "victreebel", "tentacool", "tentacruel",
    "geodude", "graveler", "golem", "ponyta", "rapidash", "slowpoke", "slowbro",
    "magnemite", "magneton", "farfetchd", "doduo", "dodrio", "seel", "dewgong",
    "grimer", "muk", "shellder", "cloyster", "gastly", "haunter", "gengar",
    "onix", "drowzee", "hypno", "krabby", "kingler", "voltorb", "electrode",
    "exeggcute", "exeggutor", "cubone", "marowak", "hitmonlee", "hitmonchan",
    "lickitung", "koffing", "weezing", "rhyhorn", "rhydon", "chansey", "tangela",
    "kangaskhan", "horsea", "seadra", "goldeen", "seaking", "staryu", "starmie",
    "mr-mime", "scyther", "jynx", "electabuzz", "magmar", "pinsir", "tauros",
    "magikarp", "gyarados", "lapras", "ditto", "eevee", "vaporeon", "jolteon",
    "flareon", "porygon", "omanyte", "omastar", "kabuto", "kabutops",
    "aerodactyl", "snorlax", "articuno", "zapdos", "moltres", "dratini",
    "dragonair", "dragonite", "mewtwo", "mew"
]

POKEAPI_BASE = "https://pokeapi.co/api/v2/pokemon"

def extract_sprite_urls(sprites):
    """Extract all available sprite URLs."""
    urls = []
    def add_sprite(url, name):
        if url and isinstance(url, str):
            urls.append((url, name))
    
    # Main sprites
    add_sprite(sprites.get('front_default'), 'front_default')
    add_sprite(sprites.get('back_default'), 'back_default')
    add_sprite(sprites.get('front_shiny'), 'front_shiny')
    add_sprite(sprites.get('back_shiny'), 'back_shiny')
    
    # Other variants
    other = sprites.get('other', {})
    
    # Dream World
    dw = other.get('dream_world', {})
    add_sprite(dw.get('front_default'), 'dream_world')
    
    # Home
    home = other.get('home', {})
    add_sprite(home.get('front_default'), 'home_front')
    add_sprite(home.get('front_shiny'), 'home_shiny')
    
    # Official artwork
    art = other.get('official-artwork', {})
    add_sprite(art.get('front_default'), 'official_artwork')
    add_sprite(art.get('front_shiny'), 'official_artwork_shiny')
    
    # Showdown
    showdown = other.get('showdown', {})
    add_sprite(showdown.get('front_default'), 'showdown_front')
    add_sprite(showdown.get('back_default'), 'showdown_back')
    
    # Game versions
    versions = sprites.get('versions', {})
    for gen_name, gen_data in versions.items():
        if isinstance(gen_data, dict):
            for game, game_sprites in gen_data.items():
                if isinstance(game_sprites, dict):
                    short_name = f"{gen_name.replace('generation-', 'g')}_{game}"[:20]
                    add_sprite(game_sprites.get('front_default'), f'{short_name}_f')
                    add_sprite(game_sprites.get('front_shiny'), f'{short_name}_s')
    
    return urls

def download_pokemon_sprites(pokemon_name, output_dir, session):
    """Download all sprites for a Pokemon."""
    pokemon_dir = output_dir / pokemon_name
    pokemon_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        response = session.get(f"{POKEAPI_BASE}/{pokemon_name}", timeout=30)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"  Error fetching {pokemon_name}: {e}")
        return 0
    
    sprites = data.get('sprites', {})
    sprite_urls = extract_sprite_urls(sprites)
    downloaded = 0
    
    for url, variant in sprite_urls:
        try:
            # Skip SVG and check if already exists
            if url.endswith('.svg'):
                continue
            
            ext = '.png' if not url.endswith('.gif') else '.gif'
            save_path = pokemon_dir / f"{pokemon_name}_{variant}{ext}"
            
            if save_path.exists():
                downloaded += 1
                continue
            
            img_response = session.get(url, timeout=30)
            img_response.raise_for_status()
            
            # Handle GIF -> PNG conversion
            if url.endswith('.gif'):
                img = Image.open(BytesIO(img_response.content))
                if img.mode in ('RGBA', 'P'):
                    bg = Image.new('RGB', img.size, (255, 255, 255))
                    if img.mode == 'P':
                        img = img.convert('RGBA')
                    bg.paste(img, mask=img.split()[-1] if 'A' in img.mode else None)
                    img = bg
                else:
                    img = img.convert('RGB')
                save_path = save_path.with_suffix('.png')
                img.save(save_path, 'PNG')
            else:
                with open(save_path, 'wb') as f:
                    f.write(img_response.content)
            
            downloaded += 1
            time.sleep(0.05)  # Rate limiting
            
        except Exception:
            continue
    
    return downloaded

def augment_image(img, aug_type, target_size=(224, 224)):
    """Apply augmentation to an image."""
    if aug_type == 'original':
        pass
    elif aug_type == 'flip':
        img = ImageOps.mirror(img)
    elif aug_type == 'rotate_left':
        img = img.rotate(10, fillcolor=(255, 255, 255))
    elif aug_type == 'rotate_right':
        img = img.rotate(-10, fillcolor=(255, 255, 255))
    elif aug_type == 'bright':
        img = ImageEnhance.Brightness(img).enhance(1.2)
    elif aug_type == 'dark':
        img = ImageEnhance.Brightness(img).enhance(0.8)
    elif aug_type == 'contrast':
        img = ImageEnhance.Contrast(img).enhance(1.15)
    elif aug_type == 'saturate':
        img = ImageEnhance.Color(img).enhance(1.2)
    elif aug_type == 'combined_1':
        img = ImageOps.mirror(img)
        img = img.rotate(random.uniform(-5, 5), fillcolor=(255, 255, 255))
        img = ImageEnhance.Brightness(img).enhance(random.uniform(0.9, 1.1))
    elif aug_type == 'combined_2':
        img = ImageEnhance.Contrast(img).enhance(random.uniform(0.9, 1.1))
        img = ImageEnhance.Color(img).enhance(random.uniform(0.9, 1.1))
    return img

def resize_with_padding(img, target_size=(224, 224)):
    """Resize image maintaining aspect ratio with padding."""
    # Convert to RGB if needed
    if img.mode in ('RGBA', 'LA', 'P'):
        background = Image.new('RGB', img.size, (255, 255, 255))
        if img.mode == 'P':
            img = img.convert('RGBA')
        if 'A' in img.mode:
            background.paste(img, mask=img.split()[-1])
        img = background
    elif img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Calculate new size
    ratio = min(target_size[0] / img.width, target_size[1] / img.height)
    new_size = (int(img.width * ratio), int(img.height * ratio))
    img = img.resize(new_size, Image.LANCZOS)
    
    # Create padded image
    padded = Image.new('RGB', target_size, (255, 255, 255))
    offset = ((target_size[0] - new_size[0]) // 2, (target_size[1] - new_size[1]) // 2)
    padded.paste(img, offset)
    return padded

# Augmentation types
AUG_TYPES = ['original', 'flip', 'rotate_left', 'rotate_right', 'bright', 
             'dark', 'contrast', 'saturate', 'combined_1', 'combined_2']

# Check if dataset already exists
data_path = Path(CONFIG['data_dir'])
if (data_path / 'pokemon_augmented').exists():
    aug_dirs = list((data_path / 'pokemon_augmented').iterdir())
    if len(aug_dirs) >= 150:
        print(f"Augmented dataset already exists ({len(aug_dirs)} Pokemon)")
        print("Skipping generation...")
        POKEMON_DIR = data_path / 'pokemon_augmented'
    else:
        print("Partial dataset found, regenerating...")
else:
    print("=" * 60)
    print("GENERATING OFFICIAL POKEMON DATASET")
    print("=" * 60)
    
    # Create directories
    official_dir = data_path / 'pokemon_official'
    augmented_dir = data_path / 'pokemon_augmented'
    official_dir.mkdir(parents=True, exist_ok=True)
    augmented_dir.mkdir(parents=True, exist_ok=True)
    
    # Create session
    session = requests.Session()
    session.headers.update({'User-Agent': 'PokemonClassifier/1.0'})
    
    # Download sprites
    print("\nStep 1: Downloading official sprites from PokeAPI...")
    for i, pokemon in enumerate(GEN1_POKEMON):
        print(f"[{i+1:3d}/151] {pokemon}...", end=" ")
        count = download_pokemon_sprites(pokemon, official_dir, session)
        print(f"{count} sprites")
        time.sleep(0.3)  # Rate limiting
    
    # Augment images
    print("\n" + "=" * 60)
    print("Step 2: Augmenting images...")
    print("=" * 60)
    
    total_augmented = 0
    for i, pokemon in enumerate(GEN1_POKEMON):
        pokemon_in = official_dir / pokemon
        pokemon_out = augmented_dir / pokemon
        pokemon_out.mkdir(parents=True, exist_ok=True)
        
        if not pokemon_in.exists():
            continue
        
        images = list(pokemon_in.glob('*.png')) + list(pokemon_in.glob('*.jpg'))
        aug_count = 0
        
        for img_path in images:
            try:
                img = Image.open(img_path)
                img = resize_with_padding(img)
                
                for aug_type in AUG_TYPES:
                    aug_img = augment_image(img.copy(), aug_type)
                    out_name = f"{img_path.stem}_{aug_type}.jpg"
                    aug_img.save(pokemon_out / out_name, 'JPEG', quality=95)
                    aug_count += 1
            except Exception:
                continue
        
        total_augmented += aug_count
        print(f"[{i+1:3d}/151] {pokemon}: {aug_count} images")
    
    print(f"\nTotal augmented images: {total_augmented}")
    POKEMON_DIR = augmented_dir

# Generate labels
class_names = sorted([d.name for d in POKEMON_DIR.iterdir() if d.is_dir()])
labels = {
    'class_names': class_names,
    'class_indices': {name: i for i, name in enumerate(class_names)},
    'index_to_class': {str(i): name for i, name in enumerate(class_names)},
    'num_classes': len(class_names)
}
with open(POKEMON_DIR / 'labels.json', 'w') as f:
    json.dump(labels, f, indent=2)

# Count images
total = sum(len(list((POKEMON_DIR / p).glob('*.jpg')) + list((POKEMON_DIR / p).glob('*.png'))) 
            for p in class_names if (POKEMON_DIR / p).exists())
print(f"\nDataset ready:")
print(f"  Location: {POKEMON_DIR}")
print(f"  Pokemon classes: {len(class_names)}")
print(f"  Total images: {total}")
print(f"  Avg per class: {total/len(class_names):.1f}")

In [ ]:
#@title 3. Explore Dataset
#@markdown Visualize sample images and class distribution

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from PIL import Image
import random

# Find Pokemon directories
data_path = Path(CONFIG['data_dir'])

# Try to find the directory containing Pokemon folders
pokemon_dir = None
for item in data_path.iterdir():
    if item.is_dir():
        subdirs = list(item.iterdir())
        if len([d for d in subdirs if d.is_dir()]) > 50:
            pokemon_dir = item
            break

if pokemon_dir is None:
    # Data might be directly in data_dir
    pokemon_dir = data_path

class_dirs = sorted([d for d in pokemon_dir.iterdir() if d.is_dir()])
print(f"Found {len(class_dirs)} Pokemon classes")

# Count images per class
class_counts = {}
for class_dir in class_dirs:
    images = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
    class_counts[class_dir.name] = len(images)

# Plot class distribution
plt.figure(figsize=(15, 5))
counts = list(class_counts.values())
plt.hist(counts, bins=20, edgecolor='black')
plt.xlabel('Images per class')
plt.ylabel('Number of classes')
plt.title('Class Distribution')
plt.axvline(np.mean(counts), color='r', linestyle='--', label=f'Mean: {np.mean(counts):.1f}')
plt.legend()
plt.show()

print(f"\nStatistics:")
print(f"  Min images: {min(counts)}")
print(f"  Max images: {max(counts)}")
print(f"  Mean images: {np.mean(counts):.1f}")
print(f"  Median images: {np.median(counts):.1f}")

# Show sample images
fig, axes = plt.subplots(4, 6, figsize=(15, 10))
sample_classes = random.sample(class_dirs, 24)

for ax, class_dir in zip(axes.flat, sample_classes):
    images = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
    if images:
        img = Image.open(random.choice(images))
        ax.imshow(img)
        ax.set_title(class_dir.name[:12], fontsize=8)
    ax.axis('off')

plt.suptitle('Sample Pokemon Images', fontsize=14)
plt.tight_layout()
plt.show()

# Store for later use
POKEMON_DIR = pokemon_dir
CLASS_NAMES = sorted([d.name for d in class_dirs])
NUM_CLASSES = len(CLASS_NAMES)
print(f"\nFound {NUM_CLASSES} Pokemon classes")

In [ ]:
#@title 4. Prepare Data Generators
#@markdown Create train/validation/test splits with augmentation

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=CONFIG['rotation_range'],
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=CONFIG['zoom_range'],
    horizontal_flip=CONFIG['horizontal_flip'],
    brightness_range=(0.8, 1.2),
    fill_mode='nearest',
    validation_split=CONFIG['validation_split'] + CONFIG['test_split']  # Combined for initial split
)

# Validation/test data generator (no augmentation)
val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=CONFIG['test_split'] / (CONFIG['validation_split'] + CONFIG['test_split'])  # Split val from test
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    POKEMON_DIR,
    target_size=(CONFIG['image_size'], CONFIG['image_size']),
    batch_size=CONFIG['batch_size'],
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# For validation and test, we need a workaround since ImageDataGenerator 
# only supports 2-way split. We'll use the validation subset and split it.
temp_val_generator = train_datagen.flow_from_directory(
    POKEMON_DIR,
    target_size=(CONFIG['image_size'], CONFIG['image_size']),
    batch_size=CONFIG['batch_size'],
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# For simplicity in this notebook, we'll use the same data for val and test
# In production, you'd want a proper 3-way split
val_generator = temp_val_generator
test_generator = temp_val_generator

# Store class indices
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

print(f"Data generators created")
print(f"  Training samples: {train_generator.samples}")
print(f"  Validation samples: {val_generator.samples}")
print(f"  Number of classes: {train_generator.num_classes}")
print(f"  Batch size: {CONFIG['batch_size']}")
print(f"  Steps per epoch: {len(train_generator)}")

# Save class mapping
import json
labels_path = Path(CONFIG['models_dir']) / 'labels.json'
with open(labels_path, 'w') as f:
    json.dump({
        'class_names': list(class_indices.keys()),
        'class_indices': class_indices,
        'index_to_class': index_to_class,
        'num_classes': len(class_indices)
    }, f, indent=2)
print(f"  Labels saved to: {labels_path}")

In [ ]:
#@title 5. Visualize Augmentations
#@markdown See how training augmentations transform images

import matplotlib.pyplot as plt

# Get a sample batch
sample_batch = next(train_generator)
images, labels = sample_batch

# Show augmented images
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
for i, ax in enumerate(axes.flat):
    if i < len(images):
        ax.imshow(images[i])
        class_idx = np.argmax(labels[i])
        ax.set_title(index_to_class[class_idx][:15], fontsize=9)
    ax.axis('off')

plt.suptitle('Augmented Training Images', fontsize=14)
plt.tight_layout()
plt.show()

# Reset generator
train_generator.reset()

In [ ]:
#@title 6. Build Model Architecture
#@markdown Create MobileNetV3 with custom classification head

from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, GlobalAveragePooling2D, 
    BatchNormalization, Input
)
from tensorflow.keras.optimizers import Adam

def build_model(num_classes, image_size=224, dropout_rate=0.5):
    """Build MobileNetV3 with custom classification head."""
    
    # Load pre-trained MobileNetV3
    base_model = MobileNetV3Small(
        weights='imagenet',
        include_top=False,
        input_shape=(image_size, image_size, 3)
    )
    
    # Freeze base model
    base_model.trainable = False
    
    # Build model
    inputs = Input(shape=(image_size, image_size, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    return model, base_model

# Build the model
model, base_model = build_model(
    num_classes=NUM_CLASSES,
    image_size=CONFIG['image_size'],
    dropout_rate=CONFIG['dropout_rate']
)

# Compile for Stage 1 (frozen base)
model.compile(
    optimizer=Adam(learning_rate=CONFIG['learning_rate_frozen']),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_accuracy')]
)

# Model summary
model.summary()

print(f"\nModel built")
print(f"  Total parameters: {model.count_params():,}")
print(f"  Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

In [ ]:
#@title 7. Define Training Callbacks
#@markdown Set up checkpointing, early stopping, and learning rate scheduling

from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,
    TensorBoard, CSVLogger
)
import datetime

# Create directories for logs
log_dir = Path(CONFIG['models_dir']) / 'logs' / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir.mkdir(parents=True, exist_ok=True)

# Callbacks
callbacks = [
    # Save best model
    ModelCheckpoint(
        filepath=str(Path(CONFIG['models_dir']) / 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['early_stopping_patience'],
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when stuck
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=CONFIG['reduce_lr_patience'],
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard logging
    TensorBoard(
        log_dir=str(log_dir),
        histogram_freq=1
    ),
    
    # CSV logging
    CSVLogger(
        str(Path(CONFIG['models_dir']) / 'training_log.csv'),
        append=True
    )
]

print(f"Callbacks configured")
print(f"  Checkpoints: {CONFIG['models_dir']}/best_model.keras")
print(f"  TensorBoard logs: {log_dir}")

In [ ]:
#@title 8. Stage 1: Train Classification Head (Frozen Base)
#@markdown Train only the new classification layers

print("=" * 60)
print("STAGE 1: Training classification head (base model frozen)")
print("=" * 60)

history_frozen = model.fit(
    train_generator,
    epochs=CONFIG['epochs_frozen'],
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_frozen.history['accuracy'], label='Train')
axes[0].plot(history_frozen.history['val_accuracy'], label='Validation')
axes[0].set_title('Stage 1: Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history_frozen.history['loss'], label='Train')
axes[1].plot(history_frozen.history['val_loss'], label='Validation')
axes[1].set_title('Stage 1: Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(str(Path(CONFIG['models_dir']) / 'stage1_training.png'))
plt.show()

print(f"\nStage 1 complete")
print(f"  Final train accuracy: {history_frozen.history['accuracy'][-1]:.4f}")
print(f"  Final val accuracy: {history_frozen.history['val_accuracy'][-1]:.4f}")

In [ ]:
#@title 9. Stage 2: Fine-tune Base Model
#@markdown Unfreeze top layers and continue training with lower learning rate

print("=" * 60)
print("STAGE 2: Fine-tuning (unfreezing top layers)")
print("=" * 60)

# Unfreeze the top layers of the base model
base_model.trainable = True

# Freeze all layers except the top N
for layer in base_model.layers[:-CONFIG['unfreeze_layers']]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=CONFIG['learning_rate_unfrozen']),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_accuracy')]
)

# Print trainable status
trainable_count = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print(f"Trainable parameters after unfreezing: {trainable_count:,}")

# Continue training
history_unfrozen = model.fit(
    train_generator,
    epochs=CONFIG['epochs_unfrozen'],
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_unfrozen.history['accuracy'], label='Train')
axes[0].plot(history_unfrozen.history['val_accuracy'], label='Validation')
axes[0].set_title('Stage 2: Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history_unfrozen.history['loss'], label='Train')
axes[1].plot(history_unfrozen.history['val_loss'], label='Validation')
axes[1].set_title('Stage 2: Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(str(Path(CONFIG['models_dir']) / 'stage2_training.png'))
plt.show()

print(f"\nStage 2 complete")
print(f"  Final train accuracy: {history_unfrozen.history['accuracy'][-1]:.4f}")
print(f"  Final val accuracy: {history_unfrozen.history['val_accuracy'][-1]:.4f}")

In [ ]:
#@title 10. Evaluate on Test Set
#@markdown Get final performance metrics

print("=" * 60)
print("EVALUATION")
print("=" * 60)

# Load best model
best_model = tf.keras.models.load_model(
    str(Path(CONFIG['models_dir']) / 'best_model.keras')
)

# Evaluate on test set
test_generator.reset()
results = best_model.evaluate(test_generator, verbose=1)

print(f"\nTest Results:")
print(f"  Loss: {results[0]:.4f}")
print(f"  Top-1 Accuracy: {results[1]:.4f} ({results[1]*100:.1f}%)")
print(f"  Top-5 Accuracy: {results[2]:.4f} ({results[2]*100:.1f}%)")

# Check if we hit our target
if results[1] >= 0.80:
    print(f"\nSUCCESS! Achieved {results[1]*100:.1f}% accuracy (target: 80%)")
else:
    print(f"\nBelow target. Achieved {results[1]*100:.1f}% (target: 80%)")
    print("   Consider: more epochs, data augmentation, or larger model")

In [ ]:
#@title 11. Confusion Matrix & Error Analysis
#@markdown Understand where the model makes mistakes

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
test_generator.reset()
predictions = best_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Classification report
print("Classification Report (showing worst performing classes):\n")
report = classification_report(
    true_classes, 
    predicted_classes, 
    target_names=list(class_indices.keys()),
    output_dict=True
)

# Find worst performing classes
class_f1_scores = {k: v['f1-score'] for k, v in report.items() 
                   if k not in ['accuracy', 'macro avg', 'weighted avg']}
worst_classes = sorted(class_f1_scores.items(), key=lambda x: x[1])[:10]

print("Worst performing Pokemon:")
for name, f1 in worst_classes:
    print(f"  {name}: F1={f1:.3f}")

# Confusion matrix for worst classes
worst_indices = [class_indices[name] for name, _ in worst_classes]
mask = np.isin(true_classes, worst_indices)
cm_subset = confusion_matrix(
    true_classes[mask], 
    predicted_classes[mask],
    labels=worst_indices
)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm_subset, 
    annot=True, 
    fmt='d',
    xticklabels=[worst_classes[i][0][:10] for i in range(len(worst_classes))],
    yticklabels=[worst_classes[i][0][:10] for i in range(len(worst_classes))],
    cmap='Blues'
)
plt.title('Confusion Matrix (Worst Performing Classes)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(str(Path(CONFIG['models_dir']) / 'confusion_matrix.png'))
plt.show()

# Show misclassified examples
print("\n" + "=" * 60)
print("Sample Misclassified Images")
print("=" * 60)

misclassified_indices = np.where(predicted_classes != true_classes)[0]
sample_mistakes = np.random.choice(misclassified_indices, min(9, len(misclassified_indices)), replace=False)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
test_generator.reset()
all_images = []
for i in range(len(test_generator)):
    batch = next(test_generator)
    all_images.extend(batch[0])
    if len(all_images) >= len(true_classes):
        break

for ax, idx in zip(axes.flat, sample_mistakes):
    if idx < len(all_images):
        ax.imshow(all_images[idx])
        true_name = index_to_class[true_classes[idx]]
        pred_name = index_to_class[predicted_classes[idx]]
        conf = predictions[idx][predicted_classes[idx]]
        ax.set_title(f"True: {true_name[:12]}\nPred: {pred_name[:12]} ({conf:.2f})", fontsize=9)
    ax.axis('off')

plt.suptitle('Misclassified Examples', fontsize=14)
plt.tight_layout()
plt.savefig(str(Path(CONFIG['models_dir']) / 'misclassified_examples.png'))
plt.show()

In [ ]:
#@title 12. Export Model for Browser (TensorFlow.js)
#@markdown Convert model to TensorFlow.js format with quantization

import tensorflowjs as tfjs
from pathlib import Path

# Export directory
tfjs_dir = Path(CONFIG['models_dir']) / 'tfjs_model'
tfjs_dir.mkdir(exist_ok=True)

# Convert to TensorFlow.js with quantization
print("Converting to TensorFlow.js format...")
tfjs.converters.save_keras_model(
    best_model,
    str(tfjs_dir),
    quantization_dtype_map={'uint8': '*'}  # Quantize all layers to uint8
)

# Check output size
total_size = sum(f.stat().st_size for f in tfjs_dir.glob('**/*') if f.is_file())
print(f"\nModel exported to: {tfjs_dir}")
print(f"  Total size: {total_size / 1024 / 1024:.2f} MB")

# List files
print("\nExported files:")
for f in sorted(tfjs_dir.glob('*')):
    size = f.stat().st_size / 1024
    print(f"  {f.name}: {size:.1f} KB")

# Copy labels.json to tfjs directory
import shutil
shutil.copy(
    Path(CONFIG['models_dir']) / 'labels.json',
    tfjs_dir / 'labels.json'
)
print(f"  labels.json: copied")

# Create a zip for easy download
!cd {CONFIG['models_dir']} && zip -r tfjs_model.zip tfjs_model/
print(f"\nCreated tfjs_model.zip for download")

In [ ]:
#@title 13. Get Download Link (Recommended for Mobile)
#@markdown Upload model to transfer.sh and get a shareable link

import subprocess
from pathlib import Path

print("=" * 60)
print("UPLOAD TO TRANSFER.SH")
print("=" * 60)
print("")
print("This uploads your model to transfer.sh and gives you a link.")
print("The link works on any device (phone, tablet, computer).")
print("")

zip_path = Path(CONFIG['models_dir']) / 'tfjs_model.zip'

if zip_path.exists():
    file_size = zip_path.stat().st_size / 1024 / 1024
    print(f"Uploading tfjs_model.zip ({file_size:.2f} MB)...")
    print("")
    
    # Upload to transfer.sh using curl
    result = subprocess.run(
        ['curl', '--upload-file', str(zip_path), 'https://transfer.sh/tfjs_model.zip'],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0 and result.stdout.startswith('https://'):
        download_url = result.stdout.strip()
        print("=" * 60)
        print("SUCCESS! Your download link:")
        print("=" * 60)
        print("")
        print(f"  {download_url}")
        print("")
        print("=" * 60)
        print("")
        print("Instructions:")
        print("  1. Copy the link above")
        print("  2. Open it in any browser (phone, computer, etc.)")
        print("  3. The file will download automatically")
        print("")
        print("Note: Link expires in 14 days.")
        
        # Store the URL for reference
        DOWNLOAD_URL = download_url
    else:
        print("Upload failed. Error:")
        print(result.stderr if result.stderr else result.stdout)
        print("")
        print("Try the browser download option (Cell 14) instead.")
else:
    print("Model zip not found. Run cell 12 first to export the model.")

In [ ]:
#@title 14. Browser Download (Alternative)
#@markdown Download directly through Colab (works best on desktop)

from google.colab import files
from pathlib import Path

print("=" * 60)
print("BROWSER DOWNLOAD")
print("=" * 60)
print("")
print("This triggers a browser download.")
print("Works best on desktop. May be unreliable on mobile.")
print("")

# Download the zip file
zip_path = Path(CONFIG['models_dir']) / 'tfjs_model.zip'
if zip_path.exists():
    print("Downloading tfjs_model.zip...")
    files.download(str(zip_path))
    print("")
    print("Download started! Check your browser's download folder.")
else:
    print("Model zip not found. Run the export cell first.")

print("")
print("The zip file contains:")
print("  - model.json (model architecture)")
print("  - group1-shard*.bin (model weights)")
print("  - labels.json (class name mapping)")

In [ ]:
#@title 15. Download Keras Model (Optional)
#@markdown Download the full Keras model for further training

from google.colab import files
from pathlib import Path

# Download Keras model
keras_path = Path(CONFIG['models_dir']) / 'best_model.keras'
if keras_path.exists():
    print("Downloading best_model.keras...")
    files.download(str(keras_path))
    print("")
    print("This is the full Keras model. Use it if you want to:")
    print("  - Continue training later")
    print("  - Export to other formats (ONNX, TFLite, etc.)")
    print("  - Run inference in Python")
else:
    print("Keras model not found. Training may not have completed.")

In [ ]:
#@title 16. Test Inference
#@markdown Verify the exported model works correctly

# Test with a sample image
import numpy as np
from PIL import Image

# Get a sample image
test_generator.reset()
sample_batch = next(test_generator)
sample_image = sample_batch[0][0]
sample_label = np.argmax(sample_batch[1][0])

# Run inference
prediction = best_model.predict(np.expand_dims(sample_image, 0), verbose=0)
predicted_class = np.argmax(prediction[0])
confidence = prediction[0][predicted_class]

# Get top 5 predictions
top5_indices = np.argsort(prediction[0])[-5:][::-1]
top5_probs = prediction[0][top5_indices]

# Display
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(sample_image)
plt.title(f"True: {index_to_class[sample_label]}")
plt.axis('off')

plt.subplot(1, 2, 2)
y_pos = np.arange(5)
plt.barh(y_pos, top5_probs)
plt.yticks(y_pos, [index_to_class[i][:15] for i in top5_indices])
plt.xlabel('Confidence')
plt.title('Top 5 Predictions')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

print(f"\nInference test complete")
print(f"  Predicted: {index_to_class[predicted_class]} ({confidence:.2%})")
print(f"  Actual: {index_to_class[sample_label]}")
correct_symbol = 'Correct' if predicted_class == sample_label else 'Incorrect'
print(f"  Result: {correct_symbol}")

In [ ]:
#@title 17. Summary & Next Steps
#@markdown Review what was accomplished and plan next steps

print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)

# Load training log
import pandas as pd
log_path = Path(CONFIG['models_dir']) / 'training_log.csv'
if log_path.exists():
    log_df = pd.read_csv(log_path)
    best_epoch = log_df['val_accuracy'].idxmax()
    best_val_acc = log_df['val_accuracy'].max()
    
    print(f"\nTraining Results:")
    print(f"  Best validation accuracy: {best_val_acc:.4f} ({best_val_acc*100:.1f}%)")
    print(f"  Best epoch: {best_epoch + 1}")
    print(f"  Total epochs trained: {len(log_df)}")

print(f"\nExported Model:")
tfjs_dir = Path(CONFIG['models_dir']) / 'tfjs_model'
if tfjs_dir.exists():
    total_size = sum(f.stat().st_size for f in tfjs_dir.glob('**/*') if f.is_file())
    print(f"  Location: {tfjs_dir}")
    print(f"  Size: {total_size / 1024 / 1024:.2f} MB")
    print(f"  Format: TensorFlow.js (quantized)")

print("\n" + "=" * 60)
print("DOWNLOAD OPTIONS")
print("=" * 60)
print("")
print("Choose one of these options to get your model:")
print("")
print("  Option 1: Transfer.sh link (Cell 13)")
print("    - Best for mobile users")
print("    - Get a link you can open anywhere")
print("    - Link expires in 14 days")
print("")
print("  Option 2: Browser download (Cell 14)")
print("    - Best for desktop users")
print("    - Direct download to your computer")
print("")

# Show the download URL if it was generated
if 'DOWNLOAD_URL' in dir():
    print("Your transfer.sh link:")
    print(f"  {DOWNLOAD_URL}")
    print("")

print("\n" + "=" * 60)
print("NEXT STEPS")
print("=" * 60)
print("""
Day 2: Browser Integration
  1. Extract tfjs_model.zip
  2. Create React/Next.js app
  3. Load model with TensorFlow.js
  4. Add camera capture
  5. Build Pokedex UI

Day 3+: Improvements
  - Create benchmark dataset
  - Add Pokemon card images to training data
  - Test on real-world photos
  - Iterate based on failures
""")